In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import datetime as dt

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings("ignore")

In [3]:
import sys
sys.path.append("../") 

from paths import make_dir_line

modality = 'e'
project = 'agrupaciones'
data = make_dir_line(modality, project)

raw = data('raw')
processed = data('processed')

# AIMS

1. Antiguedad de la cuenta: Dias desde la creacion de la cuenta hasta la fecha.
2. Ciclo de vida: Dias desde primer compra a ultima compra.
3. Tiempo promedio y mediana entre compras: Dias promedio o mediana entre compras
4. Costo de vida: Costo de vida
5. Clientes activos
6. Frecuente 2 y 3
7. Recuperados
8. Perdidos

## 1. Antiguedad de la cuenta

In [4]:
compras = pd.read_parquet(processed / 'compras.parquet.gzip')
compras = compras.sort_values(by=['fecha'], ascending=False)
compras = compras.groupby('cliente').head(1).reset_index()
compras['antiguedad'] = (dt.datetime.now() - compras['fecha']).dt.days
compras

,index,cliente,monto,cuotas,fecha,dias_mora,antiguedad
0,7,Ana,300,2,2023-02-01,7,93
1,1,Juan,5000,24,2022-02-01,0,458
2,5,Pedro,2000,12,2022-02-01,0,458


## 2. Ciclo de vida

In [5]:
compras = pd.read_parquet(processed / 'compras.parquet.gzip')

In [6]:
# crear una nueva columna con la fecha de inicio del ciclo de vida de cada cliente
fecha_inicio = compras.groupby('cliente')['fecha'].min()
fecha_inicio

cliente
Ana     2022-01-01
Juan    2022-01-01
Pedro   2022-01-01
Name: fecha, dtype: datetime64[ns]

In [7]:
# agregar el inicio del ciclo de vida
compras['inicio_ciclo_vida'] = compras['cliente'].map(fecha_inicio)
compras

,cliente,monto,cuotas,fecha,dias_mora,inicio_ciclo_vida
2,Ana,2000,6,2022-01-01,25,2022-01-01
3,Ana,3000,12,2022-03-01,0,2022-01-01
6,Ana,5000,24,2022-03-01,8,2022-01-01
7,Ana,300,2,2023-02-01,7,2022-01-01
0,Juan,1000,12,2022-01-01,0,2022-01-01
1,Juan,5000,24,2022-02-01,0,2022-01-01
4,Pedro,1500,8,2022-01-01,11,2022-01-01
5,Pedro,2000,12,2022-02-01,0,2022-01-01


In [8]:
# crear una nueva columna con la fecha de fin del ciclo de vida de cada cliente
fecha_fin = compras.groupby('cliente')['fecha'].max()
fecha_fin

cliente
Ana     2023-02-01
Juan    2022-02-01
Pedro   2022-02-01
Name: fecha, dtype: datetime64[ns]

In [9]:
compras['fin_ciclo_vida'] = compras['cliente'].map(fecha_fin)
compras

,cliente,monto,cuotas,fecha,dias_mora,inicio_ciclo_vida,fin_ciclo_vida
2,Ana,2000,6,2022-01-01,25,2022-01-01,2023-02-01
3,Ana,3000,12,2022-03-01,0,2022-01-01,2023-02-01
6,Ana,5000,24,2022-03-01,8,2022-01-01,2023-02-01
7,Ana,300,2,2023-02-01,7,2022-01-01,2023-02-01
0,Juan,1000,12,2022-01-01,0,2022-01-01,2022-02-01
1,Juan,5000,24,2022-02-01,0,2022-01-01,2022-02-01
4,Pedro,1500,8,2022-01-01,11,2022-01-01,2022-02-01
5,Pedro,2000,12,2022-02-01,0,2022-01-01,2022-02-01


In [10]:
# calcular la duración del ciclo de vida de cada cliente
compras['ciclo_vida'] = (pd.to_datetime(compras['fin_ciclo_vida']) - pd.to_datetime(compras['inicio_ciclo_vida'])).dt.days
compras

,cliente,monto,cuotas,fecha,dias_mora,inicio_ciclo_vida,fin_ciclo_vida,ciclo_vida
2,Ana,2000,6,2022-01-01,25,2022-01-01,2023-02-01,396
3,Ana,3000,12,2022-03-01,0,2022-01-01,2023-02-01,396
6,Ana,5000,24,2022-03-01,8,2022-01-01,2023-02-01,396
7,Ana,300,2,2023-02-01,7,2022-01-01,2023-02-01,396
0,Juan,1000,12,2022-01-01,0,2022-01-01,2022-02-01,31
1,Juan,5000,24,2022-02-01,0,2022-01-01,2022-02-01,31
4,Pedro,1500,8,2022-01-01,11,2022-01-01,2022-02-01,31
5,Pedro,2000,12,2022-02-01,0,2022-01-01,2022-02-01,31


In [11]:
compras = compras.groupby('cliente').head(1).reset_index()
compras

,index,cliente,monto,cuotas,fecha,dias_mora,inicio_ciclo_vida,fin_ciclo_vida,ciclo_vida
0,2,Ana,2000,6,2022-01-01,25,2022-01-01,2023-02-01,396
1,0,Juan,1000,12,2022-01-01,0,2022-01-01,2022-02-01,31
2,4,Pedro,1500,8,2022-01-01,11,2022-01-01,2022-02-01,31


## 3. Tiempo promedio y mediana entre creditos

In [12]:
compras = pd.read_parquet(processed / 'compras.parquet.gzip')

In [13]:
# agrupar los créditos por ID de cliente
grupo_clientes = compras.groupby('cliente')
grupo_clientes

In [14]:
# calcular la diferencia en días entre cada crédito y el siguiente crédito
diferencia_dias = grupo_clientes['fecha'].diff().dt.days
diferencia_dias

2      NaN
3     59.0
6      0.0
7    337.0
0      NaN
1     31.0
4      NaN
5     31.0
Name: fecha, dtype: float64

In [15]:
# calcular el promedio de la diferencia en días para cada cliente
promedio_dias = diferencia_dias.groupby(compras['cliente']).mean()
promedio_dias

cliente
Ana      132.0
Juan      31.0
Pedro     31.0
Name: fecha, dtype: float64

In [16]:
# calcular la mediana de la diferencia en días para cada cliente
promedio_dias = diferencia_dias.groupby(compras['cliente']).median()
promedio_dias

cliente
Ana      59.0
Juan     31.0
Pedro    31.0
Name: fecha, dtype: float64

## 4. Costo de vida

In [17]:
compras = pd.read_parquet(processed / 'compras.parquet.gzip')

In [18]:
# agrupar las transacciones del cliente por ID
grupo_clientes = compras.groupby('cliente')
grupo_clientes

In [19]:
# calcular la fecha de la última transacción y el tiempo transcurrido desde la primera
fecha_ultima_trans = grupo_clientes['fecha'].max()
tiempo_transcurrido = (fecha_ultima_trans - grupo_clientes['fecha'].min()).dt.days

In [20]:
# calcular el número de transacciones del cliente y el valor promedio de la transacción
num_transacciones = grupo_clientes['monto'].count()
valor_promedio = grupo_clientes['monto'].mean()

In [21]:
# definir la tasa de descuento
tasa_descuento = 0.1

In [22]:
# calcular el CLV utilizando la fórmula de VPN
clv = (valor_promedio * num_transacciones * (1 - np.exp(-tasa_descuento * tiempo_transcurrido))) / tasa_descuento

# imprimir el resultado
print(clv)

cliente
Ana      103000.000000
Juan      57297.047856
Pedro     33423.277916
dtype: float64


## 5. Clientes activos

In [50]:
compras = pd.read_parquet(processed / 'compras.parquet.gzip')

new_date = dt.datetime(2019,1,1)
nueva_fila = {'cliente': 'eduardo', 'monto': 4000, 'cuotas': 10, 'fecha':new_date, 'dias_mora':0}
compras = compras.append(nueva_fila, ignore_index=True)

compras.head(10)

,cliente,monto,cuotas,fecha,dias_mora
0,Ana,2000,6,2022-01-01,25
1,Ana,3000,12,2022-03-01,0
2,Ana,5000,24,2022-03-01,8
3,Ana,300,2,2023-02-01,7
4,Juan,1000,12,2022-01-01,0
5,Juan,5000,24,2022-02-01,0
6,Pedro,1500,8,2022-01-01,11
7,Pedro,2000,12,2022-02-01,0
8,eduardo,4000,10,2019-01-01,0


In [51]:
fecha_max = compras['fecha'].max()
fecha_max

Timestamp('2023-02-01 00:00:00')

In [56]:
df_activos = compras[compras['fecha'] >= fecha_max - pd.DateOffset(years=1)]
df_activos.head()

,cliente,monto,cuotas,fecha,dias_mora
1,Ana,3000,12,2022-03-01,0
2,Ana,5000,24,2022-03-01,8
3,Ana,300,2,2023-02-01,7
5,Juan,5000,24,2022-02-01,0
7,Pedro,2000,12,2022-02-01,0


In [53]:
clientes_ultimo_anio = df_activos['cliente'].unique()
clientes_ultimo_anio

array(['Ana', 'Juan', 'Pedro'], dtype=object)

In [57]:
compras['activos'] = np.where(compras['cliente'].isin(clientes_ultimo_anio), 1, 0)
compras

,cliente,monto,cuotas,fecha,dias_mora,activos
0,Ana,2000,6,2022-01-01,25,1
1,Ana,3000,12,2022-03-01,0,1
2,Ana,5000,24,2022-03-01,8,1
3,Ana,300,2,2023-02-01,7,1
4,Juan,1000,12,2022-01-01,0,1
5,Juan,5000,24,2022-02-01,0,1
6,Pedro,1500,8,2022-01-01,11,1
7,Pedro,2000,12,2022-02-01,0,1
8,eduardo,4000,10,2019-01-01,0,0


## 6. Frecuente 2 y 3

In [122]:
compras = pd.read_parquet(processed / 'compras.parquet.gzip')

new_date = dt.datetime(2022,5,1)
nueva_fila = {'cliente': 'juan', 'monto': 4000, 'cuotas': 10, 'fecha':new_date, 'dias_mora':0}
compras = compras.append(nueva_fila, ignore_index=True)
nueva_fila = {'cliente': 'juan', 'monto':700, 'cuotas': 2, 'fecha':new_date, 'dias_mora':66}
compras = compras.append(nueva_fila, ignore_index=True)

new_date = dt.datetime(2019,1,1)
nueva_fila = {'cliente': 'eduardo', 'monto': 4000, 'cuotas': 10, 'fecha':new_date, 'dias_mora':0}
compras = compras.append(nueva_fila, ignore_index=True)

compras.head(10)

,cliente,monto,cuotas,fecha,dias_mora
0,Ana,2000,6,2022-01-01,25
1,Ana,3000,12,2022-03-01,0
2,Ana,5000,24,2022-03-01,8
3,Ana,300,2,2023-02-01,7
4,Juan,1000,12,2022-01-01,0
5,Juan,5000,24,2022-02-01,0
6,Pedro,1500,8,2022-01-01,11
7,Pedro,2000,12,2022-02-01,0
8,juan,4000,10,2022-05-01,0
9,juan,700,2,2022-05-01,66


In [123]:
fecha_max = compras['fecha'].max()
fecha_max

Timestamp('2023-02-01 00:00:00')

In [124]:
ventas_ultimo_anio = compras[compras['fecha'] >= fecha_max - pd.DateOffset(years=1)]

ventas_por_cliente = ventas_ultimo_anio.groupby('cliente').count()
clientes_2_compras = ventas_por_cliente[ventas_por_cliente['fecha'] == 2]

clientes_2_compras = clientes_2_compras.reset_index()
clientes_2_compras = clientes_2_compras[['cliente']]
clientes_2_compras = ventas_ultimo_anio.merge(clientes_2_compras, on='cliente')
clientes_2_compras

,cliente,monto,cuotas,fecha,dias_mora
0,juan,4000,10,2022-05-01,0
1,juan,700,2,2022-05-01,66


In [125]:
clientes_dos_compras = clientes_2_compras['cliente'].unique()
clientes_dos_compras

array(['juan'], dtype=object)

In [126]:
compras['frecuentes_2'] = np.where(compras['cliente'].isin(clientes_dos_compras), 1, 0)
compras

,cliente,monto,cuotas,fecha,dias_mora,frecuentes_2
0,Ana,2000,6,2022-01-01,25,0
1,Ana,3000,12,2022-03-01,0,0
2,Ana,5000,24,2022-03-01,8,0
3,Ana,300,2,2023-02-01,7,0
4,Juan,1000,12,2022-01-01,0,0
5,Juan,5000,24,2022-02-01,0,0
6,Pedro,1500,8,2022-01-01,11,0
7,Pedro,2000,12,2022-02-01,0,0
8,juan,4000,10,2022-05-01,0,1
9,juan,700,2,2022-05-01,66,1


In [127]:
clientes_3_compras = ventas_por_cliente[ventas_por_cliente['fecha'] == 3]

clientes_3_compras = clientes_3_compras.reset_index()
clientes_3_compras = clientes_3_compras[['cliente']]
clientes_3_compras = ventas_ultimo_anio.merge(clientes_3_compras, on='cliente')
clientes_3_compras

,cliente,monto,cuotas,fecha,dias_mora
0,Ana,3000,12,2022-03-01,0
1,Ana,5000,24,2022-03-01,8
2,Ana,300,2,2023-02-01,7


In [128]:
clientes_tres_compras = clientes_3_compras['cliente'].unique()
clientes_tres_compras

array(['Ana'], dtype=object)

In [129]:
compras['frecuentes_3'] = np.where(compras['cliente'].isin(clientes_tres_compras), 1, 0)
compras

,cliente,monto,cuotas,fecha,dias_mora,frecuentes_2,frecuentes_3
0,Ana,2000,6,2022-01-01,25,0,1
1,Ana,3000,12,2022-03-01,0,0,1
2,Ana,5000,24,2022-03-01,8,0,1
3,Ana,300,2,2023-02-01,7,0,1
4,Juan,1000,12,2022-01-01,0,0,0
5,Juan,5000,24,2022-02-01,0,0,0
6,Pedro,1500,8,2022-01-01,11,0,0
7,Pedro,2000,12,2022-02-01,0,0,0
8,juan,4000,10,2022-05-01,0,1,0
9,juan,700,2,2022-05-01,66,1,0


## 7. Recuperados

In [143]:
new_date = dt.datetime(2022,5,1)

compras = pd.read_parquet(processed / 'compras.parquet.gzip')
nueva_fila = {'cliente': 'juan', 'monto': 4000, 'cuotas': 10, 'fecha':new_date, 'dias_mora':0}
compras = compras.append(nueva_fila, ignore_index=True)
nueva_fila = {'cliente': 'juan', 'monto':700, 'cuotas': 2, 'fecha':new_date, 'dias_mora':66}
compras = compras.append(nueva_fila, ignore_index=True)

new_date = dt.datetime(2019,1,1)
nueva_fila = {'cliente': 'hector', 'monto':5500, 'cuotas': 17, 'fecha':new_date, 'dias_mora':1}
compras = compras.append(nueva_fila, ignore_index=True)

new_date = dt.datetime(2023,1,1)
nueva_fila = {'cliente': 'valentina', 'monto':5500, 'cuotas': 17, 'fecha':new_date, 'dias_mora':1}
compras = compras.append(nueva_fila, ignore_index=True)

new_date = dt.datetime(2021,12,1)
nueva_fila = {'cliente': 'valentina', 'monto':600, 'cuotas': 6, 'fecha':new_date, 'dias_mora':1}
compras = compras.append(nueva_fila, ignore_index=True)

new_date = dt.datetime(2019,1,1)
nueva_fila = {'cliente': 'eduardo', 'monto': 4000, 'cuotas': 10, 'fecha':new_date, 'dias_mora':0}
compras = compras.append(nueva_fila, ignore_index=True)

compras.head(20)

,cliente,monto,cuotas,fecha,dias_mora
0,Ana,2000,6,2022-01-01,25
1,Ana,3000,12,2022-03-01,0
2,Ana,5000,24,2022-03-01,8
3,Ana,300,2,2023-02-01,7
4,Juan,1000,12,2022-01-01,0
5,Juan,5000,24,2022-02-01,0
6,Pedro,1500,8,2022-01-01,11
7,Pedro,2000,12,2022-02-01,0
8,juan,4000,10,2022-05-01,0
9,juan,700,2,2022-05-01,66


In [144]:
fecha_max = compras['fecha'].max()
fecha_max

Timestamp('2023-02-01 00:00:00')

In [154]:
clientes_ultimo_anio = compras[compras['fecha'] >= fecha_max - pd.DateOffset(years=1)]['cliente'].unique()
clientes_ultimo_anio

array(['Ana', 'Juan', 'Pedro', 'juan', 'valentina'], dtype=object)

In [156]:
ventas_ultimo_anio = compras[compras['cliente'].isin(clientes_ultimo_anio)]
ventas_ultimo_anio = ventas_ultimo_anio.sort_values(['cliente', 'fecha'], ascending=False)
ventas_ultimo_anio

,cliente,monto,cuotas,fecha,dias_mora
11,valentina,5500,17,2023-01-01,1
12,valentina,600,6,2021-12-01,1
8,juan,4000,10,2022-05-01,0
9,juan,700,2,2022-05-01,66
7,Pedro,2000,12,2022-02-01,0
6,Pedro,1500,8,2022-01-01,11
5,Juan,5000,24,2022-02-01,0
4,Juan,1000,12,2022-01-01,0
3,Ana,300,2,2023-02-01,7
1,Ana,3000,12,2022-03-01,0


In [157]:
ventas_por_cliente = ventas_ultimo_anio.groupby('cliente')
ventas_por_cliente

In [158]:
ultimas_fechas = ventas_por_cliente.apply(lambda x: x['fecha'].tail(2))
ultimas_fechas

cliente      
Ana        2    2022-03-01
           0    2022-01-01
Juan       5    2022-02-01
           4    2022-01-01
Pedro      7    2022-02-01
           6    2022-01-01
juan       8    2022-05-01
           9    2022-05-01
valentina  11   2023-01-01
           12   2021-12-01
Name: fecha, dtype: datetime64[ns]

In [171]:
diferencias = ultimas_fechas.groupby('cliente').diff()
diferencias = diferencias.reset_index()
diferencias['fecha'] = diferencias['fecha'].dt.days
clientes_seleccionados = diferencias[diferencias['fecha'] <= -365]
clientes_seleccionados = clientes_seleccionados[['cliente']]
clientes_seleccionados

,cliente
9,valentina


In [172]:
clientes_seleccionados = ventas_ultimo_anio.merge(clientes_seleccionados, on='cliente')
clientes_seleccionados

,cliente,monto,cuotas,fecha,dias_mora
0,valentina,5500,17,2023-01-01,1
1,valentina,600,6,2021-12-01,1


In [173]:
clientes_recuperados = clientes_seleccionados['cliente'].unique()
clientes_recuperados

array(['valentina'], dtype=object)

In [174]:
compras['recuperados'] = np.where(compras['cliente'].isin(clientes_recuperados), 1, 0)
compras

,cliente,monto,cuotas,fecha,dias_mora,recuperados
0,Ana,2000,6,2022-01-01,25,0
1,Ana,3000,12,2022-03-01,0,0
2,Ana,5000,24,2022-03-01,8,0
3,Ana,300,2,2023-02-01,7,0
4,Juan,1000,12,2022-01-01,0,0
5,Juan,5000,24,2022-02-01,0,0
6,Pedro,1500,8,2022-01-01,11,0
7,Pedro,2000,12,2022-02-01,0,0
8,juan,4000,10,2022-05-01,0,0
9,juan,700,2,2022-05-01,66,0


## 8. Perdidos

In [175]:
new_date = dt.datetime(2022,5,1)

compras = pd.read_parquet(processed / 'compras.parquet.gzip')
nueva_fila = {'cliente': 'juan', 'monto': 4000, 'cuotas': 10, 'fecha':new_date, 'dias_mora':0}
compras = compras.append(nueva_fila, ignore_index=True)
nueva_fila = {'cliente': 'juan', 'monto':700, 'cuotas': 2, 'fecha':new_date, 'dias_mora':66}
compras = compras.append(nueva_fila, ignore_index=True)

new_date = dt.datetime(2019,1,1)
nueva_fila = {'cliente': 'hector', 'monto':5500, 'cuotas': 17, 'fecha':new_date, 'dias_mora':1}
compras = compras.append(nueva_fila, ignore_index=True)

new_date = dt.datetime(2023,1,1)
nueva_fila = {'cliente': 'valentina', 'monto':5500, 'cuotas': 17, 'fecha':new_date, 'dias_mora':1}
compras = compras.append(nueva_fila, ignore_index=True)

new_date = dt.datetime(2021,12,1)
nueva_fila = {'cliente': 'valentina', 'monto':600, 'cuotas': 6, 'fecha':new_date, 'dias_mora':1}
compras = compras.append(nueva_fila, ignore_index=True)

new_date = dt.datetime(2019,1,1)
nueva_fila = {'cliente': 'eduardo', 'monto': 4000, 'cuotas': 10, 'fecha':new_date, 'dias_mora':0}
compras = compras.append(nueva_fila, ignore_index=True)

compras.head(20)

,cliente,monto,cuotas,fecha,dias_mora
0,Ana,2000,6,2022-01-01,25
1,Ana,3000,12,2022-03-01,0
2,Ana,5000,24,2022-03-01,8
3,Ana,300,2,2023-02-01,7
4,Juan,1000,12,2022-01-01,0
5,Juan,5000,24,2022-02-01,0
6,Pedro,1500,8,2022-01-01,11
7,Pedro,2000,12,2022-02-01,0
8,juan,4000,10,2022-05-01,0
9,juan,700,2,2022-05-01,66


In [176]:
fecha_max = compras['fecha'].max()
fecha_max

Timestamp('2023-02-01 00:00:00')

In [177]:
clientes_ultimos_dos_anios = compras[compras['fecha'] >= fecha_max - pd.DateOffset(years=2)]['cliente'].unique()
clientes_ultimos_dos_anios

array(['Ana', 'Juan', 'Pedro', 'juan', 'valentina'], dtype=object)

In [178]:
todos_clientes = compras['cliente'].unique()
todos_clientes

array(['Ana', 'Juan', 'Pedro', 'juan', 'hector', 'valentina', 'eduardo'],
      dtype=object)

In [181]:
clientes_sin_compra = list(set(todos_clientes) - set(clientes_ultimos_dos_anios))
clientes_sin_compra

['hector', 'eduardo']

In [182]:
compras['perdidos'] = np.where(compras['cliente'].isin(clientes_sin_compra), 1, 0)
compras

,cliente,monto,cuotas,fecha,dias_mora,perdidos
0,Ana,2000,6,2022-01-01,25,0
1,Ana,3000,12,2022-03-01,0,0
2,Ana,5000,24,2022-03-01,8,0
3,Ana,300,2,2023-02-01,7,0
4,Juan,1000,12,2022-01-01,0,0
5,Juan,5000,24,2022-02-01,0,0
6,Pedro,1500,8,2022-01-01,11,0
7,Pedro,2000,12,2022-02-01,0,0
8,juan,4000,10,2022-05-01,0,0
9,juan,700,2,2022-05-01,66,0


In [24]:
print('ok_')

ok_
